## Part 1

In [11]:
mutable struct Code
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    eop::Bool
end

str2prog(s) = parse.(Int, split(s, ","))

Code(s::String) = Code(str2prog(s), 1, [], [], false)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(code::Code) = Instruction(code.code[code.cur])

function apply(code::Code, instruction::Instruction)
    if instruction.op == 99
        code.eop = true
    elseif instruction.op == 1
        a1 = instruction.modes[1] == 1 ? code.code[code.cur + 1] : code.code[code.code[code.cur + 1] + 1]
        a2 = instruction.modes[2] == 1 ? code.code[code.cur + 2] : code.code[code.code[code.cur + 2] + 1]
        code.code[code.code[code.cur + 3] + 1] = a1 + a2
        code.cur += 4
    elseif instruction.op == 2
        a1 = instruction.modes[1] == 1 ? code.code[code.cur + 1] : code.code[code.code[code.cur + 1] + 1]
        a2 = instruction.modes[2] == 1 ? code.code[code.cur + 2] : code.code[code.code[code.cur + 2] + 1]
        code.code[code.code[code.cur + 3] + 1] = a1 * a2
        code.cur += 4
    elseif instruction.op == 3
        code.code[code.code[code.cur + 1] + 1] = popfirst!(code.input)
        code.cur += 2
    elseif instruction.op == 4
        push!(code.output, instruction.modes[1] == 1 ? code.code[code.cur + 1] : code.code[code.code[code.cur + 1] + 1])
        code.cur += 2
    elseif instruction.op == 5
        a1 = instruction.modes[1] == 1 ? code.code[code.cur + 1] : code.code[code.code[code.cur + 1] + 1]
        a2 = instruction.modes[2] == 1 ? code.code[code.cur + 2] : code.code[code.code[code.cur + 2] + 1]
        if a1 != 0 
            code.cur = a2 + 1 
        else
            code.cur += 3
        end
    elseif instruction.op == 6
        a1 = instruction.modes[1] == 1 ? code.code[code.cur + 1] : code.code[code.code[code.cur + 1] + 1]
        a2 = instruction.modes[2] == 1 ? code.code[code.cur + 2] : code.code[code.code[code.cur + 2] + 1]
        if a1 == 0 
            code.cur = a2 + 1 
        else
            code.cur += 3
        end
    elseif instruction.op == 7
        a1 = instruction.modes[1] == 1 ? code.code[code.cur + 1] : code.code[code.code[code.cur + 1] + 1]
        a2 = instruction.modes[2] == 1 ? code.code[code.cur + 2] : code.code[code.code[code.cur + 2] + 1]
        a3 = code.code[code.cur + 3] + 1
        if a1 < a2
            code.code[a3] = 1
        else
            code.code[a3] = 0
        end
        code.cur += 4
    elseif instruction.op == 8
        a1 = instruction.modes[1] == 1 ? code.code[code.cur + 1] : code.code[code.code[code.cur + 1] + 1]
        a2 = instruction.modes[2] == 1 ? code.code[code.cur + 2] : code.code[code.code[code.cur + 2] + 1]
        a3 = code.code[code.cur + 3] + 1
        if a1 == a2
            code.code[a3] = 1
        else
            code.code[a3] = 0
        end
        code.cur += 4
    end
end

function run(code::Code, input::Vector{Int})
    code.input = input
    while !code.eop
        instruction = Instruction(code)
        apply(code, instruction)
    end
    
    code.output
end

run(code) = run(code, [])

run (generic function with 2 methods)

In [20]:
function amplout(s, phases)
    state = 0
    for i in phases
        code = Code(s)
        out = run(code, [i, state])
        state = out[1]
    end
    
    state
end

@assert amplout("3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0", [4, 3, 2, 1, 0]) == 43210
@assert amplout("3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0", [0,1,2,3,4]) == 54321
@assert amplout("3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0", [1,0,4,3,2]) == 65210

In [28]:
function perms(acc, head, tails)
    if isempty(tails)
       push!(acc, head)
    else
        for (i, tail) in enumerate(tails)
            perms(acc, vcat(head, tail), vcat(tails[1:(i - 1)], tails[(i+1): end]))
        end
    end
end

perms (generic function with 1 method)

In [37]:
function findmaxphase(s)
    states = Vector{Vector{Int}}()
    perms(states, Int[], [0, 1, 2, 3, 4])
    maximum(map(x -> amplout(s, x), states))
end

findmaxphase (generic function with 1 method)

In [39]:
@assert findmaxphase("3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0") == 43210
@assert findmaxphase("3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0") == 54321
@assert findmaxphase("3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0") == 65210

In [41]:
println(findmaxphase(readline("input.txt")))

880726


In [24]:
for (i, j) in enumerate([10, 20])
    println(i, ":", j)
end

1:10
2:20


In [6]:
?popfirst!

search: popfirst!



```
popfirst!(collection) -> item
```

Remove the first `item` from `collection`.

# Examples

```jldoctest
julia> A = [1, 2, 3, 4, 5, 6]
6-element Array{Int64,1}:
 1
 2
 3
 4
 5
 6

julia> popfirst!(A)
1

julia> A
5-element Array{Int64,1}:
 2
 3
 4
 5
 6
```
